# Feature Translation Service SWORD River Data Demo

This notebook shows a few example requests to the Feature Translation Service (FTS) Application Programming Interface (API).

## Required Dependencies

In order to have the interactive map widget work, the "jupyter-leaflet" extension must be added to the JupyterLab instance that is running this notebook. Instructions for installation can be found:

https://ipyleaflet.readthedocs.io/en/latest/installation.html#jupyterlab-extension

Alternatively, if your instance of JupyterLab has the "Extension Manager" enabled, you may be able to simply search for "leaflet" and install the extension that way. Typically a page reload is required after installation.

In addition to the JupyterLab widget, there are two python dependencies that must be available to the python kernel running this notebook. 

- ipyleaflet (https://ipyleaflet.readthedocs.io/en/latest/)
- geojson (https://pypi.org/project/geojson/)

The next cell installs them when the cell is run.

In [1]:
!pip install ipyleaflet geojson

  Using cached ipyleaflet-0.14.0-py2.py3-none-any.whl (3.3 MB)
  Using cached geojson-2.5.0-py2.py3-none-any.whl (14 kB)
  Using cached ipywidgets-7.6.3-py2.py3-none-any.whl (121 kB)
  Using cached jupyterlab_widgets-1.0.0-py3-none-any.whl (243 kB)
  Attempting uninstall: ipywidgets
    Found existing installation: ipywidgets 7.5.1
    Uninstalling ipywidgets-7.5.1:
      Successfully uninstalled ipywidgets-7.5.1


In [9]:
import requests
import json
import geojson
from ipyleaflet import Map, GeoJSON, FullScreenControl
from ipywidgets import Layout
from IPython.display import JSON

def response_to_FeatureCollection(response):
    """
    This function will return a geojson.FeatureCollection representation of the features found
    in the provided response.
    Parameters
    ----------
    response : requests.Response
        Response object returned from a GET request on the FTS rivers endpoint.
    Returns
    -------
    geojson.FeatureCollection
        FeatureCollection containing all features extracted from the response.
    """
    featureList = []
    for reach_id, reach_json in response.json()['results'].items():
        reach_feature = geojson.loads(json.dumps(reach_json['geojson']))
        reach_feature['properties']={k:v for k,v in reach_json.items() if k not in ['geojson', 'geometry']}
        featureList.append(reach_feature)
    featureCollection = geojson.FeatureCollection(featureList)
    return featureCollection

def estimate_center_of_FeatureCollection(featureCollection):
    """
    This function does a very simplistic estimation of the center of all features in the given FeatureCollection.
    Parameters
    ----------
    featureCollection : geojson.FeatureCollection
        Estimate the center lon, lat of this FeatureCollection.
    Returns
    -------
    tuple(float, float)
        Estimated center longitude, center latitude
    """
    lats = [xy[1] for feature in featureCollection['features'] for xy in feature['coordinates']]
    lons = [xy[0] for feature in featureCollection['features'] for xy in feature['coordinates']]

    center_lat = (min(lats) + max(lats)) / 2
    center_lon = (min(lons) + max(lons)) / 2
    
    return center_lon, center_lat

In [12]:
response = requests.get("https://fts.podaac.earthdata.nasa.gov/rivers/reach/74230900271")

featureCollection = response_to_FeatureCollection(response)

JSON(reaches.json())

<IPython.core.display.JSON object>

In [15]:
center_lon, center_lat = estimate_center_of_FeatureCollection(featureCollection)
m = Map(center=(center_lat, center_lon), zoom=13, layout=Layout(width='90%', height='1000px'))
m.add_control(FullScreenControl())

geo_json = GeoJSON(
    data=featureCollection
)

m.add_layer(geo_json)
m

Map(center=[33.377498, -91.10319899999999], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_i…

In [16]:
response = requests.get("https://fts.podaac.earthdata.nasa.gov/rivers/reach/74230900")

featureCollection = response_to_FeatureCollection(response)

JSON(reaches.json())

<IPython.core.display.JSON object>

In [19]:
center_lon, center_lat = estimate_center_of_FeatureCollection(featureCollection)
m = Map(center=(center_lat, center_lon), zoom=9, layout=Layout(width='90%', height='1000px'))
m.add_control(FullScreenControl())

geo_json = GeoJSON(
    data=featureCollection
)

m.add_layer(geo_json)
m

Map(center=[33.059585, -91.0671805], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title…

In [20]:
response = requests.get("https://fts.podaac.earthdata.nasa.gov/rivers/node/7423090027")

featureCollection = response_to_FeatureCollection(response)

JSON(nodes.json())

<IPython.core.display.JSON object>

In [22]:
lats = [feature['coordinates'][1] for feature in featureCollection['features']]
lons = [feature['coordinates'][0] for feature in featureCollection['features']]

center_lat = (min(lats) + max(lats)) / 2
center_lon = (min(lons) + max(lons)) / 2

m = Map(center=(center_lat, center_lon), zoom=13, layout=Layout(width='90%', height='1000px'))
m.add_control(FullScreenControl())

geo_json = GeoJSON(
    data=featureCollection
)

m.add_layer(geo_json)
m

Map(center=[33.377269, -91.1028605], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title…